In [6]:
import tensorflow as tf

In [7]:
class Config:
    img_h = 32
    img_w = 32
    img_c = 3
    num_classes = 100
    
    learning_rate = 1e-3
    summary_dir = './exp_1'

In [8]:
class Cifar:
    
    def __init__(self, config):
        self.config = config
    
    def init_input(self):
        with tf.variable_scope('inputs'):
            self.x = tf.placeholder(tf.float32, [None, self.config.img_h, self.config.img_w, self.config.img_c], name='X')
            self.y = tf.placeholder(tf.int64, [None], name='Y')
            self.is_training = tf.placeholder(tf.bool, name='Training_flag')
            tf.add_to_collection('inputs',self.x)
            tf.add_to_collection('inputs',self.y)
            tf.add_to_collection('inputs',self.is_training)
                
    def init_network(self):
        with tf.variable_scope('network'):
            conv1 = Cifar.conv_bn_relu('conv1_block', self.x, 16, (3, 3), self.is_training)
            conv2 = Cifar.conv_bn_relu('conv2_block', conv1, 32, (3, 3), self.is_training)
            
            with tf.variable_scope('max_pool1'):
                max_pool1 = tf.layers.max_pooling2d(conv2, pool_size=(2,2), strides=(2,2), name='max_pool')
            
            conv3 = Cifar.conv_bn_relu('conv3_block', max_pool1, 32, (3,3), self.is_training)
            conv4 = Cifar.conv_bn_relu('conv4_block', conv3, 32, (3,3), self.is_training)
            
            with tf.variable_scope('max_pool2'):
                max_pool2 = tf.layers.max_pooling2d(conv4, pool_size=(2,2), strides=(2,2), name='max_pool')
                
            with tf.variable_scope('flatten'):
                flattened = tf.layers.flatten(max_pool2, name='flatten')
                
            dense1 = Cifar.dense_bn_relu_dropout('dense1', flattened, 512, 0.5, self.is_training)
            dense2 = Cifar.dense_bn_relu_dropout('dense2', dense1, 256, 0.3, self.is_training)
            
            with tf.variable_scope('out'):
                self.out = tf.layers.dense(dense2, self.config.num_classes,
                                           kernel_initializer=tf.initializers.truncated_normal, name='out')
                tf.add_to_collection('out',self.out)

    
    def init_ops(self):
        with tf.variable_scope('out_argmax'):
            self.out_argmax = tf.argmax(self.out, axis=-1, output_type=tf.int64, name='out_argmax')
        
        with tf.variable_scope('loss-acc'):
            self.loss = tf.losses.sparse_softmax_cross_entropy(labels=self.y, logits=self.out)
            self.acc = tf.metrics.accuracy(labels=self.y, predictions=self.out_argmax)            
        
        with tf.variable_scope('train_step'):
            self.optimizer = tf.train.AdamOptimizer(self.config.learning_rate)
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                self.train_step = self.optimizer.minimize(self.loss)
    
    def build(self):
        self.init_input()
        self.init_network()
        self.init_ops()
        
    @staticmethod
    def conv_bn_relu(name, x, out_filters, kernel_size, training_flag):
        with tf.variable_scope(name):
            out = tf.layers.conv2d(x, out_filters, kernel_size, padding='SAME', 
                                   kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv')
            out = tf.layers.batch_normalization(out, training=training_flag, name='bn')
            out = tf.nn.relu(out)
            return out
            
    
    @staticmethod
    def dense_bn_relu_dropout(name, x, num_neurons, dropout_rate, training_flag):
        with tf.variable_scope(name):
            out = tf.layers.dense(x, num_neurons, kernel_initializer=tf.initializers.truncated_normal, name='dense')
            out = tf.layers.batch_normalization(out, training=training_flag, name='bn')
            out = tf.nn.relu(out)
            out = tf.layers.dropout(out, dropout_rate, training=training_flag, name='dropout')
            return out
        

In [9]:
tf.reset_default_graph()

sess = tf.Session()

model = Cifar(Config)
model.build()

print('Model is built successfully')

import os

if not os.path.exists(Config.summary_dir):
    os.makedirs(Config.summary_dir)
    
summary_writer = tf.summary.FileWriter(Config.summary_dir, graph=sess.graph)



Model is built successfully


In [21]:
init = tf.global_variables_initializer().run(session=sess)

print(tf.get_collection('inputs'))
x, y, is_training = tf.get_collection('inputs')
out = tf.get_collection('out')
print(x)
print(y)
print(is_training)
print(out)
import numpy as np
out = out[0]
out_np = sess.run([out], feed_dict={x:np.zeros((1,32,32,3)),is_training:False})
print(out_np)
print(out_np[0].shape)
print(out_np[0].dtype)

[<tf.Tensor 'inputs/X:0' shape=(?, 32, 32, 3) dtype=float32>, <tf.Tensor 'inputs/Y:0' shape=(?,) dtype=int64>, <tf.Tensor 'inputs/Training_flag:0' shape=<unknown> dtype=bool>]
Tensor("inputs/X:0", shape=(?, 32, 32, 3), dtype=float32)
Tensor("inputs/Y:0", shape=(?,), dtype=int64)
Tensor("inputs/Training_flag:0", dtype=bool)
[<tf.Tensor 'network/out/out/BiasAdd:0' shape=(?, 100) dtype=float32>]
[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)]
(1, 100)
float32
